# Семинар 4: нейронная сеть

*Задание основано на ноутбуке курса Школы Анализа Данных по глубинному обучению.*

На этом семинаре мы напишем нейронную сеть с нуля. Реализуемая нами архитектура в целом схожа с тем, как выглядит нейронная сеть в Pytorch.

Этот семинар состоит из двух ноутбуков: Modules.ipynb, в котором будет находиться реализация нейронной сети, и этот ноутбук, в котором вы будете выполнять все эксперименты с нейронной сетью. Пожалуйста, не меняйте прототипы функций.

## Введение

Нейронные сети стали популярными по многим причинам, но одна из них - это модульность. Нейронные сети состоят из модулей (слоев), каждый слой реализует какую-то функциональность. Комбинируя имеющиеся слои можно реализовать state-of-art архитектуру с помощью уже имеющейся библиотеки (Pytorch, Tensorflow итд). Часто для реализации множества прорывных современных идей достаточно определить новый слой, или даже просто слегка изменить уже имеющийся.

<img src=img/simple_neural_network_header.jpg>

Давайте для начала посмотрим на нейронную сеть как на черный ящик (нас не интересует как он устроен, но когда мы просим его что-то сделать - он вежливо выполняет просьбу). Какую функциональность должна иметь нейронная сеть? Такую же как и остальные модели машинного обучения, а именно:

1) По данному входу (input) она должна выдавать предсказания (output)

2) Она должна быть обучаемой (уметь подстраиваться под имеющиеся данные)

Остановимся пока на первом пункте. Назовем метод, который по данному входу дает какие-то предсказания **forward** (если вы делали предыдущее домашнее задание - то это в точности метод **forward_pass** из него). Обратите внимание что метод **forward** должен не только возвращать значение output, но и сохранять его в поле self.output.




<img src=img/black-box.png>

В ноутбуке modules.ipynb вы найдете несколько различных классов:

**Module** - этот самый описанный выше черный ящик. Пока что вас в нем интересует только метод **forward** который по данному input считает необходимый output. Все остальные модули отнаследованы от этого класса.

**Sequential** - это класс-контейнер. Он состоит из списка различных модулей. Метод forward в нем последовательно прогоняет input через каждый модуль: сначала input подается в первый модуль, затем output первого модуля подается как input второго и так далее.

**Sigmoid** - это активационная функция. Ее метод forward для каждого элемента матрицы input считает значение сигмоиды в нем. output имеет ту же размерность, что и input.

**ReLU** - это активационная функция. Ее метод forward для каждого элемента матрицы input считает значение ReLU в нем. output имеет ту же размерность, что и input.

**Linear** - это линейный слой (без активационной функции!). При инициализации он принимает пару чисел - свою размерность (n_out) и размерность предыдущего слоя (n_in), и инициализирует свои веса $W$ случайным образом. Обратите внимание, чтo размерность матрицы весов $W$ это (n_out, n_in), то есть $W_{ji}$ - это вес соединения между i-м нейроном предыдущего слоя и j-м нейроном текущего слоя. Его метод forward вычисляет значение матрицы $output$ по формуле $$output_{ij} = \sum_n input_{in} W_{jn},$$ где i - индекс объекта, а j - индекс нейрона текущего слоя.

In [ ]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

## Dumb forward

Реализуйте метод dumb_forward в `Modules.ipynb` для модуля Linear.

In [ ]:
%run modules.ipynb
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

Протестируйте ваш dumb_forward на искуственно сгенерированных данных.

In [ ]:

input_size = 4
num_classes = 3
num_inputs = 5

def init_linear():
    np.random.seed(0)
    return Linear(input_size, num_classes)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    
    # y закодирован с помощью one hot encoding, 
    y = np.zeros((5,3))
    y[np.arange(5), [0, 1, 2, 2, 1]] = 1
    return X, y

linear = init_linear()
X, y = init_toy_data()

In [ ]:
output = linear.dumb_forward(X)
print('Your output:')
print(output)
print()
print('correct output:')
correct_output = np.asarray(
[[ -1.54788446,  -6.00658097,   6.39369587],
 [ -3.07885716,  -8.08969546,  11.56531411],
 [  0.19697038,  -9.59100515,   5.43998673],
 [ -0.31239372,  -5.33085678,  1.94239605],
 [ -1.66825993,   1.10786278,   0.51528511]])
print(correct_output)
print()

print('Difference between your scores and correct scores:')
print(rel_error(output, correct_output))

# Forward

Реализуйте метод forward в модуле Linear с помощью матричных операций библиотеки numpy и протестируйте его на тех же данных.

In [ ]:
%run modules.ipynb

output = linear.forward(X)
print('Your output:')
print(output)
print()
print('correct output:')
print(correct_output)
print()

print('Difference between your scores and correct scores:')
print(rel_error(output, correct_output))

Давайте посчитаем выход двухслойной сети, значение для которой мы считали на лекции.

In [ ]:
sequential = Sequential()
linear_small = Linear(n_in=2, n_out=2)
linear_small.W = np.array([[0.9, 0.3], [0.2, 0.8]])
sequential.add(linear_small)
sequential.add(Sigmoid())

In [ ]:
X_small = np.array([[1, 0.5]])
output_small = sequential.forward(X_small)
print('Your output:')
print(output_small)
print()
print('correct output:')
correct_output_small = np.asarray([[0.7408, 0.6457]])
print(correct_output_small)
print()

print('Difference between your scores and correct scores:')
print(rel_error(output_small, correct_output_small))

# Backward

Полкчив какие-то предсказание от нейронной сети, мы хотим понять две вещи:

1) Насколько она ошиблась

2) Как надо поменять веса чтобы уменьшить это ошибку

Для подсчета ошибки в modules.ipynb реализован еще один абстрактный класс - класс **Criterion**, черный ящик соответствующий функции потерь. У него есть метод forward, который принимает два аргумента: предсказанные значения и истинные значения, и возвращает значение функции потерь для данной пары. Также реализована среднеквадратичная функция потерь - **MSECriterion**.

<img src='img/Criterion.png' width=500pt>

Мы хотим обучаться методом градиентного спуска. Для этого нам нужно посчитать частные производные функции потерь по всем параметрам модели. Мы будем делать это методом обратного распространения ошибки, в основе которого лежит следующая идея: зная градиент функции потерь по выходу модуля мы легко можем посчитать градиент функции потерь по параметрам модуля и по его входу.

<img src='img/Backprop.png' width=500>

$$\frac{\partial L}{\partial input_i} = \sum_n \frac{\partial L}{\partial output_n}\frac{\partial output_n}{\partial input_i}, \frac{\partial L}{\partial parameter} = \sum_n \frac{\partial L}{\partial output_n}\frac{\partial output_n}{\partial parameter}$$

В частности для линейного слоя имеем формулы $$\frac{\partial L}{\partial input_{ik}}= \sum_n \frac{\partial L}{\partial output_{in}}W_{nk},\frac{\partial L}{\partial W_{nk}}= \sum_i\frac{\partial L}{\partial output_{in}}input_{k}$$

где i - это индекс объекта.

За подсчет градиентов функции ошибки по входу модуля и по параметрам модуля отвечают методы **accGradParameters** и **updateGradInput**, каждый из них на вход принимает input, использованный в методе forward ранее, и gradOutput, матрица производных функции потерь по каждому из выходов (имеет ту же размерность, что и output модуля). 

**accGradParameters** считает матрицу производных функции потерь по параметрам модуля и сохраняет ее.

**updateGradInput** считает матрицу производных функции потерь по входу модуля и сохраняет ее в поле self.gradInput.

Реализуйте методы updateGradInput и accGradParameters для модуля Linear.

In [ ]:
%run modules.ipynb

# Используйте численное дифференцирования чтобы проверить вашу реализацию подсчета градиента
# Если ваша реализация верна, то относительная ошибка будет не больше 1e-5

linear = init_linear()
output = linear.forward(X)

mse = MSECriterion()
grad_output = mse.updateGradInput(output, y)

def calc_numerical_grad_for_linear(X=X, linear=linear, eps=1e-8):
    n_objects, input_size = X.shape
    n_classes, _ = linear.W.shape
    W = linear.W.copy()
    W_grad = np.zeros_like(W)
    for row_idx in range(num_classes):
        for column_idx in range(input_size):
            linear.W = W.copy()
            linear.W[row_idx][column_idx] += eps
            right_output = mse.forward(linear.forward(X), y)
            linear.W = W.copy()
            linear.W[row_idx][column_idx] -= eps
            left_output = mse.forward(linear.forward(X), y)
            W_grad[row_idx][column_idx] = (right_output - left_output) / (2 * eps)
    X_grad = np.zeros_like(X)
    for obj_idx in range(n_objects):
        for column_idx in range(input_size):
            right_X = X.copy()
            right_X[obj_idx][column_idx] += eps
            right_output = mse.forward(linear.forward(right_X), y)
            left_X = X.copy()
            left_X[obj_idx][column_idx] -= eps
            left_output = mse.forward(linear.forward(left_X), y)
            X_grad[obj_idx][column_idx] = (right_output - left_output) / (2 * eps)
    return W_grad, X_grad

W_grad_numerical, X_grad_numerical = calc_numerical_grad_for_linear()
X_grad = linear.backward(X, grad_output)
W_grad = linear.getGradParameters()[0]

print('Your gradient w.r.t W:')
print(W_grad)
print()
print('correct gradient w.r.t W:')
print(W_grad_numerical)
print()

print('Difference between your scores and correct scores:')
print(rel_error(W_grad, W_grad_numerical))
print('Your gradient w.r.t input:')
print(X_grad)
print()
print('correct gradient w.r.t input:')
print(X_grad_numerical)
print()

print('Difference between your scores and correct scores:')
print(rel_error(X_grad, X_grad_numerical))